In [1]:
# importing required modules
import pandas as pd 
import mysql.connector as msql
import math
import os
import re

In [2]:
# Importing Excel file form directory 
directory_path =(r'C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\04 Incubator - Curriculum Cohort\Incubator 7.0 TSWRDC\Source\Genral Info')

excel_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

for file in excel_files:
    file_path = os.path.join(directory_path, file)
    df = pd.read_csv(file_path)
    print(f"Data from {file}:")

Data from TS_TT_General Info - Inc 7.0.csv:


In [3]:
df.head(5)

,Sno.,Student Code,Name of the Student,Form_Name,Student Role,Batch type,Gender,Currently Pursuing year,Subject Area,Name of the college,Email,Assigned through,Incubator Course Name,Incubator Batch
0,1,SS240070001,A AMULYA,A AMULYA-SS240070001,Student,Inc_07,Female,II nd year,Pharmacy,"TSWRD Pharmacy College, Mahbubabad",atelliamulya@gmail.com,Scholarship,Kalpana-She for STEM (TS/TT/BC) : Feb - Sep 20...,INC007
1,2,SS240070002,T ANUSHA,T ANUSHA-SS240070002,Student,Inc_07,Female,II nd year,Pharmacy,"TSWRD Pharmacy College, Mahbubabad",anushathurpati37@gmail.com,Scholarship,Kalpana-She for STEM (TS/TT/BC) : Feb - Sep 20...,INC007
2,3,SS240070003,P BABY,P BABY-SS240070003,Student,Inc_07,Female,II nd year,Pharmacy,"TSWRD Pharmacy College, Mahbubabad",pulibaby408@gmail.com,Scholarship,Kalpana-She for STEM (TS/TT/BC) : Feb - Sep 20...,INC007
3,4,SS240070004,K BHARGAVI,K BHARGAVI-SS240070004,Student,Inc_07,Female,II nd year,Pharmacy,"TSWRD Pharmacy College, Mahbubabad",karlapudibhargavi00@gmail.com,Scholarship,Kalpana-She for STEM (TS/TT/BC) : Feb - Sep 20...,INC007
4,6,SS240070006,M GANGA,M GANGA-SS240070006,Student,Inc_07,Female,II nd year,Pharmacy,"TSWRD Pharmacy College, Mahbubabad",gangamacherla@gmail.com,Scholarship,Kalpana-She for STEM (TS/TT/BC) : Feb - Sep 20...,INC007


In [4]:
#Reading She for STEM Incubator file present on source files
directory_path =(r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\04 Incubator - Curriculum Cohort\Incubator 7.0 TSWRDC\Source\Course Report")
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(directory_path, file)
    data = pd.read_csv(file_path)
    print(f"Data from {file}:")

Data from Feb - Sep 2025 Batch-1741585075931.csv:


In [5]:
df.columns

Index(['Sno.', 'Student Code', 'Name of the Student', 'Form_Name',
       'Student Role', 'Batch type ', 'Gender ', 'Currently Pursuing year',
       'Subject Area', 'Name of the college', 'Email', 'Assigned through',
       'Incubator Course Name', 'Incubator Batch'],
      dtype='object')

In [6]:
# Define a list of column names to extract for putting into database 
columns_to_extract = ['Name', 'Email', 'Segment', 'Mobile', 'Enroll Date', 'Assigned Through', 'Course Name','Start Date','Last Login']

# Use the loc method to extract the specified columns
data = data.loc[:, columns_to_extract]

In [7]:
# Few rows of data
data.shape

(1813, 9)

In [8]:
df.columns

Index(['Sno.', 'Student Code', 'Name of the Student', 'Form_Name',
       'Student Role', 'Batch type ', 'Gender ', 'Currently Pursuing year',
       'Subject Area', 'Name of the college', 'Email', 'Assigned through',
       'Incubator Course Name', 'Incubator Batch'],
      dtype='object')

In [9]:
# Function to clean email addresses
def clean_email(email):

    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
df['Email'] = df['Email'].apply(clean_email)

In [10]:
# Column Names of data
df.columns

Index(['Sno.', 'Student Code', 'Name of the Student', 'Form_Name',
       'Student Role', 'Batch type ', 'Gender ', 'Currently Pursuing year',
       'Subject Area', 'Name of the college', 'Email', 'Assigned through',
       'Incubator Course Name', 'Incubator Batch'],
      dtype='object')

In [11]:
# Renaming Column name in data
df.columns =['Email', 'Name', 'Phone', 'Gender', 'Currently_Pursuing_Degree',
       'Subject_Area', 'Name_of_College_University', 'Country',
       'State_Union_Territory', 'District', 'City_Category', 'Caste_Category',
       'Annual_Family_Income', 'Assigned_Through','Incubator_Course_Name','Incubator_Batch']

ValueError: Length mismatch: Expected axis has 14 elements, new values have 16 elements

In [ ]:
df.columns

# General Information MySQL table

In [ ]:
# Connecting to MySQL Database
conn= msql.connect(host='localhost',user='root',password="VS@123",database="she_for_stem",auth_plugin='mysql_native_password')


In [ ]:
cursor =conn.cursor()

In [ ]:
for i, row in df.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    columns = ','.join(df.columns)
    placeholders = ','.join(['%s']*len(row))
    # Construct the INSERT query with ON DUPLICATE KEY UPDATE clause
    query = f"INSERT INTO she_for_stem.01_general_information ({columns}) VALUES ({placeholders}) ON DUPLICATE KEY UPDATE "
    query += ", ".join([f"{col}=VALUES({col})" for col in df.columns if col != 'Email'])
    # Execute the query
    cursor.execute(query, tuple(row))

In [ ]:
# Inserting records from data file, if already present ignoring it.
for i,row in df.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    cursor.execute("INSERT IGNORE INTO she_for_stem.01_general_information(Email, Name, Phone, Gender, Currently_Pursuing_Degree, Subject_Area, Name_of_College_University, Country, State_Union_Territory, District, City_Category, Caste_Category, Annual_Family_Income, Assigned_Through,Incubator_Course_Name,Incubator_Batch) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", tuple(row))


In [ ]:
conn.commit()

# Payment details MySQL table

In [12]:
# Creating new table which for Payment which is taken from incubator graphy sheet
Payment=pd.DataFrame(df[["Email","Assigned through"]])

In [17]:
# Rename a single column
Payment = Payment.rename(columns={'Assigned through': 'Assigned_Through'})

In [18]:
# Assigning fee cost to different cathegory of enrollment
Payment["Assigned_Through"]=Payment["Assigned_Through"].replace(['Scholarship'],'0')

In [20]:
Payment

,Email,Assigned_Through
0,atelliamulya@gmail.com,0
1,anushathurpati37@gmail.com,0
2,pulibaby408@gmail.com,0
3,karlapudibhargavi00@gmail.com,0
4,gangamacherla@gmail.com,0
...,...,...
1820,nikithamachukuri84@gmail.com,0
1821,arrollahyma28@gmail.com,0
1822,chakalivrithika@gmail.com,0
1823,janagamhemalatha85@gmail.com,0


In [21]:
# Connecting to MySQL Database
conn= msql.connect(host='localhost',user='root',password="VS@123",database="she_for_stem",auth_plugin='mysql_native_password')
cursor =conn.cursor()

In [22]:
# Inserting data into 02_payment_details 
for i,row in Payment.iterrows():
    cursor.execute("insert Ignore into 02_payment_details (Email,Incubator_Fee) values(%s,%s)",tuple(row))

In [23]:
# Replacing data with new data into 02_payment_details 
for i,row in Payment.iterrows():
    cursor.execute("REPLACE into 02_payment_details (Email,Incubator_Fee) values(%s,%s)",tuple(row))

In [24]:
conn.commit()

# Enroll Date MySQL Table

In [25]:
# Creating new table which for Enroll_Dates which is taken from incubator graphy sheet
Enroll=pd.DataFrame(data[["Email", 'Enroll Date']])

In [26]:
# Extracting only Enroll_Date
Enroll[['Enroll Date','Time']]=Enroll['Enroll Date'].str.split(' ',expand=True)
Enroll=Enroll.drop(["Time"],axis=1)

In [27]:
# Connecting to MySQL Database
conn= msql.connect(host='localhost',user='root',password="VS@123",database="she_for_stem",auth_plugin='mysql_native_password')
cursor =conn.cursor()

In [28]:
# Inserting data to 03_enroll_date table
for i,row in Enroll.iterrows():
    cursor.execute("insert IGNORE into 03_enroll_date (Email,Incubator) values(%s,%s)",tuple(row))

In [29]:
# Replacing data  with new data to 03_enroll_date table
for i,row in Enroll.iterrows():
    cursor.execute("REPLACE into 03_enroll_date (Email,Incubator) values(%s,%s)",tuple(row))

In [30]:
conn.commit()

# Batch Lastlogin Startdate MySQL Table

In [31]:
# Creating new table which for Batch which is taken from incubator graphy sheet
Batch=pd.DataFrame(data[["Email", 'Last Login','Start Date']])

In [32]:
Batch

,Email,Last Login,Start Date
0,sreejithsreenivasan2147@yahoo.co.in,08/03/25 10:33,Not Started
1,sreejith.sreenivasan@vigyanshaala.com,06/03/25 08:53,2025/02/14
2,salini.1405@gmail.com,10/03/25 04:57,Not Started
3,sreejith.sreenivasan2147@gmail.com,05/03/25 11:12,2025/02/19
4,shubham.jaybhaye@vigyanshaala.com,08/03/25 04:34,Not Started
...,...,...,...
1808,bhukyashravani2006@gmail.com,-,Not Started
1809,gugulothukalpana2004@gmail.com,-,Not Started
1810,jaruplajyothi17@gmail.com,-,Not Started
1811,nagamallurakshitha23@proton.me,-,Not Started


In [33]:
# Extracting only Enroll_Date
Batch[['Last Login','Time']]=Batch['Last Login'].str.split(' ',expand=True)
Batch=Batch.drop(["Time"],axis=1)

In [34]:
# Function to convert 'Start Date' format
def convert_start_date(date_str):
    if re.match(r'\d{4}/\d{2}/\d{2}', date_str):
        return pd.to_datetime(date_str, format='%Y/%m/%d').strftime('%d-%m-%Y')
    return date_str

# Function to convert 'Last Login' format
def convert_last_login(date_str):
    if re.match(r'\d{2}/\d{2}/\d{2}', date_str):
        return pd.to_datetime(date_str, format='%d/%m/%y').strftime('%d-%m-%Y')
    return date_str


# Apply the conversion functions to the respective columns
Batch['Start Date'] = Batch['Start Date'].apply(convert_start_date)
Batch['Last Login'] = Batch['Last Login'].apply(convert_last_login)

In [35]:
# Connecting to MySQL Database
conn= msql.connect(host='localhost',user='root',password="VS@123",database="she_for_stem",auth_plugin='mysql_native_password')
cursor =conn.cursor()

In [36]:
# Inserting data to 15_batch_lastlogin_startdate table
for i,row in Batch.iterrows():
    cursor.execute("insert IGNORE into 15_batch_lastlogin_startdate (Email,Incubator_Last_Login,Incubator_Start_Date) values(%s,%s,%s)",tuple(row))

In [37]:
# Replacing data  with new data to 03_enroll_date table
for i,row in Batch.iterrows():
    cursor.execute("REPLACE into 15_batch_lastlogin_startdate (Email,Incubator_Last_Login,Incubator_Start_Date) values(%s,%s,%s)",tuple(row))

In [38]:
conn.commit()